# Pre-processing

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('data/spotify_songs.csv')

In [ ]:
display(data)
display(data.describe().T.style.background_gradient(cmap='YlGnBu'))

In [ ]:
# check for missing values
missing_values = data.isnull().sum()
display(missing_values)

# display the lines with missing values
missing_data = data[data.isnull().any(axis=1)]
display(missing_data)

In [ ]:
# Remove id columns
data = data.drop(columns=['track_id', 'track_album_id', "playlist_id"], axis=1)
# drop the missing values
data = data.dropna()
display(data)


In [ ]:
# transform track_album_release_date into datetime
data['track_album_release_date'] = pd.to_datetime(data['track_album_release_date'], format='mixed')

# transform categorical columns into categorical data type
categorical_cols = ['playlist_genre', 'playlist_subgenre', 'track_artist', 'playlist_name', 'track_album_name']
for col in categorical_cols:
    data[col] = data[col].astype('category')

# transform the duration_ms into minutes
data['duration_s'] = data['duration_ms'] / 1000
data.drop(columns=['duration_ms'], inplace=True)

# For numeric columns that represent discrete values (like key and mode), convert to categorical
key_mapping = {
    0: 'C', 1: 'C♯/D♭', 2: 'D', 3: 'D♯/E♭', 4: 'E', 5: 'F',
    6: 'F♯/G♭', 7: 'G', 8: 'G♯/A♭', 9: 'A', 10: 'A♯/B♭', 11: 'B'
}
data['key'] = data['key'].map(key_mapping).astype('category')

data['mode'] = data['mode'].map({0: 'Minor', 1: 'Major'}).astype('category')



In [ ]:
data.info()

# Descriptive Analysis

# PCA

In [ ]:
import numpy as np
import prince

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Select only the quantitative columns

qualisup = 'playlist_genre'


data_quanti = data.select_dtypes(include=['int64', 'float64'])
data_quanti[qualisup] = data[qualisup]
data_quanti = data_quanti.set_index(qualisup)




# display(data_quanti.head())

pca = prince.PCA(
    n_components=4,
    n_iter=3,
    rescale_with_mean=True,
    rescale_with_std=True,
    copy=True,
    check_input=True,
    engine='sklearn',
    random_state=1
)
pca = pca.fit(
    data_quanti,
    sample_weight=None,
    column_weight=None,
    supplementary_columns=None
)



In [ ]:
pca.scree_plot()

In [ ]:
plt.boxplot(pca.row_coordinates(data_quanti))

In [ ]:
correlations = pca.column_correlations
plt.figure(figsize=(10, 8))
sns.heatmap(correlations, annot=True, cmap='RdBu', center=0)
plt.title('Corrélations entre les variables et les composantes principales')
plt.show()


### Interprétation de l'Analyse en Composantes Principales (PCA)


#### **Analyse des principales corrélations :**
- **PC1 (Composante Principale 1) :**  
  - **Corrélations négatives fortes :** `energy (-0.91)`, `loudness (-0.80)`.  
    → Cette composante sépare les morceaux énergiques et bruyants des morceaux plus doux.  
  - **Corrélation positive forte :** `acousticness (+0.72)`.  
    → Elle oppose également les morceaux acoustiques aux morceaux énergétiques.  

- **PC2 (Composante Principale 2) :**  
  - **Corrélations négatives :** `danceability (-0.68)`, `valence (-0.62)`.  
    → Cette composante distingue les morceaux dansants et joyeux des autres.  
  - **Corrélations positives :** `instrumentalness (+0.45)`, `duration_s (+0.38)`.  
    → Elle représente les morceaux plus longs et instrumentaux.  

- **PC3 (Composante Principale 3) :**  
  - **Corrélations positives :** `danceability (+0.46)`, `instrumentalness (+0.39)`, `duration_s (+0.45)`.  
    → Cette dimension semble être liée à la dansabilité et à la durée des morceaux.  
  - **Corrélation négative :** `tempo (-0.35)`.  
    → Elle oppose les morceaux rapides et lents.  

- **PC4 (Composante Principale 4) :**  
  - **Corrélation positive forte :** `speechiness (+0.71)`.  
    → Cette composante est dominée par la présence de paroles (ex : rap).  
  - **Corrélation positive :** `liveness (+0.58)`.  
    → Elle met en avant les morceaux enregistrés en live.  

### **Interprétation globale :**
- **PC1 sépare les morceaux énergiques et bruyants des morceaux acoustiques.**  
- **PC2 distingue les morceaux dansants et joyeux des morceaux instrumentaux et longs.**  
- **PC3 différencie les morceaux en fonction de leur dansabilité et de leur tempo.**  
- **PC4 met en avant les morceaux avec beaucoup de paroles et ceux enregistrés en live.**  



In [ ]:
# %load solutions/pmca/plot_pca.py
def plot_pca(ax1=0, ax2=1, pca=pca, data=data_quanti):
  dataset = pca.transform(data)
  dataset.reset_index(inplace=True)
  sns.scatterplot(data = dataset,
                  x = ax1, y = ax2,
                  hue = qualisup, alpha=.3)

  plt.xlabel('Component {} — {:.2f}%'.format(ax1, pca.percentage_of_variance_[ax1]))
  plt.ylabel('Component {} — {:.2f}%'.format(ax2, pca.percentage_of_variance_[ax2]))
  plt.grid(True)
  plt.show()

plot_pca(0,1)
plot_pca(1,2)
plot_pca(0,2)